# 1. Load data

In [1]:
import pandas as pd

In [2]:
cleanData = "simple.csv"
data = pd.read_csv(cleanData, sep=',')

In [3]:
data = data.ix[:, ['parcelid', 'latitude', 'longitude']]

In [4]:
data.head()

,parcelid,latitude,longitude
0,10754147,34144442,-118654084
1,10759547,34140430,-118625364
2,10843547,33989359,-118394633
3,10859147,34148863,-118437206
4,10879947,34194168,-118385816


In [5]:
data.size

3000

# 2. Pack rows into items

In [6]:
for attribute in data:
    print(attribute)

parcelid
latitude
longitude


In [5]:
for index, item in data.iterrows():
    if (index % 25 == 24):
        print(index, item['parcelid'], item['latitude'], item['longitude'])

24 11417147 33931500 -118352104
49 11818747 34072919 -118164004


In [6]:
# pick up the first 25 rows as a sample dataset
sample = data[0:25]
print(sample)

    parcelid  latitude  longitude
0   10754147  34144442 -118654084
1   10759547  34140430 -118625364
2   10843547  33989359 -118394633
3   10859147  34148863 -118437206
4   10879947  34194168 -118385816
5   10898347  34171873 -118380906
6   10933547  34131929 -118351474
7   10940747  34171345 -118314900
8   10954547  34218210 -118331311
9   10976347  34289776 -118432085
10  11073947  34265214 -118520217
11  11114347  34447747 -118565056
12  11116947  34465048 -118568166
13  11142747  34416889 -118505805
14  11193347  34585014 -118162010
15  11215747  34563376 -118019104
16  11229347  34526913 -118050581
17  11287347  34690736 -118135225
18  11288547  34733960 -118139298
19  11324547  34560018 -118169806
20  11391347  33986910 -118329553
21  11395747  33960238 -118319986
22  11404347  33959896 -118350438
23  11405747  33952952 -118361711
24  11417147  33931500 -118352104


In [9]:
size = len(sample)
print(size)

25


In [10]:
for row in sample.itertuples():
    if (row[0] == size - 1):
        print(row[0])

24


In [11]:
row 

Pandas(Index=24, parcelid=11417147, latitude=33931500, longitude=-118352104)

In [12]:
for attribute in row:
    print(attribute)

24
11417147
33931500
-118352104


### 2.1 How to build a json format item

In [13]:
test = {
        'Name': '10759547',
        'Attributes': [
            {
                'Name': 'latitude',
                'Value': '34140430',
                'Replace': True
            },
            {
                'Name': 'longitude',
                'Value': '-118625364',
                'Replace': True
            },
        ]
    }

In [14]:
group = []

In [15]:
group.append(test)

In [16]:
group

[{'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '34140430'},
   {'Name': 'longitude', 'Replace': True, 'Value': '-118625364'}],
  'Name': '10759547'}]

In [17]:
test2 = {
        'Name': '00000',
        'Attributes': [
            {
                'Name': 'latitude',
                'Value': '00000',
                'Replace': True
            },
            {
                'Name': 'longitude',
                'Value': '00000',
                'Replace': True
            },
        ]
    }

In [18]:
group.append(test2)

In [19]:
group

[{'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '34140430'},
   {'Name': 'longitude', 'Replace': True, 'Value': '-118625364'}],
  'Name': '10759547'},
 {'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '00000'},
   {'Name': 'longitude', 'Replace': True, 'Value': '00000'}],
  'Name': '00000'}]

### 2.2 Test generating items using sample dataset

In [20]:
# generate one item
def generate_one_item(row):
    # index, parcelid, latitude, longitude
    parcelid = str(row[1])
    latitude = str(row[2])
    longitude = str(row[3])
    item = {
                'Name': parcelid,
                'Attributes': [
                    {
                        'Name': 'latitude',
                        'Value': latitude,
                        'Replace': True
                    },
                    {
                        'Name': 'longitude',
                        'Value': longitude,
                        'Replace': True
                    },
                ]
            }
    return item

In [21]:
generate_one_item(row)

{'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '33931500'},
  {'Name': 'longitude', 'Replace': True, 'Value': '-118352104'}],
 'Name': 'nan'}

### 2.3 Test Putting Function

In [22]:
# put_items function test
def put_items(sdb, domainName, data):
    numOfItems = len(data)
    items_per_batch = []
    for row in data.itertuples():
        items_per_batch.append(generate_one_item(row))
        # put 25 items each time
        if (row[0] % 25 == 24) or (row[0] == numOfItems - 1):
            put_items_one_batch(sdb, domainName, items_per_batch)
            # reset items to []
            # items_per_batch = []
    return items_per_batch
            
# Do nothing here just for testing
def put_items_one_batch(sdb, domainName, items_per_batch):
#     response = sdb.batch_put_attributes(
#         DomainName=domainName,
#         Items=items_per_batch
#     )

    return

In [23]:
sdb = None
domainName = "ZillowData"
items_per_batch = put_items(sdb, domainName, sample)

In [24]:
print("Start: " + str(row[0] // 25 + 1) + " Put")

Start: 1 Put


In [25]:
type(items_per_batch)

list

# 3. Sample queries connecting to DBaas

In [133]:
import json
import boto3
import pandas as pd
from pandas import DataFrame
from geopy.distance import vincenty

In [134]:
# Config.json location
configFile = 'config.json'
with open(configFile) as globalSettings:
    config = json.load(globalSettings)

# read keys
ACCESS_KEY = str(config['AWSAccess'])
SECRET_KEY = str(config['AWSSecret'])
region = str(config['region'])

In [135]:
# Connect to SDB
Session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=region
)
sdb = Session.client('sdb')

In [136]:
response = sdb.select(
    SelectExpression="select * from ZillowData limit 1000",
#     NextToken='string',
#     ConsistentRead=True|False
)
# pick up the 'Items' part from json structure
result = response['Items']#[3]['Attributes'][0]['Value']
result

[{'Attributes': [{'Name': 'latitude', 'Value': '34144442'},
   {'Name': 'longitude', 'Value': '-118654084'}],
  'Name': '10754147'},
 {'Attributes': [{'Name': 'latitude', 'Value': '34140430'},
   {'Name': 'longitude', 'Value': '-118625364'}],
  'Name': '10759547'},
 {'Attributes': [{'Name': 'latitude', 'Value': '33989359'},
   {'Name': 'longitude', 'Value': '-118394633'}],
  'Name': '10843547'},
 {'Attributes': [{'Name': 'latitude', 'Value': '34148863'},
   {'Name': 'longitude', 'Value': '-118437206'}],
  'Name': '10859147'},
 {'Attributes': [{'Name': 'latitude', 'Value': '34194168'},
   {'Name': 'longitude', 'Value': '-118385816'}],
  'Name': '10879947'},
 {'Attributes': [{'Name': 'latitude', 'Value': '34171873'},
   {'Name': 'longitude', 'Value': '-118380906'}],
  'Name': '10898347'},
 {'Attributes': [{'Name': 'latitude', 'Value': '34131929'},
   {'Name': 'longitude', 'Value': '-118351474'}],
  'Name': '10933547'},
 {'Attributes': [{'Name': 'latitude', 'Value': '34171345'},
   {'Name

# 4. Compute the 10 closest loactions

In [137]:
# set the default location
curLat = 34
curLong = -118
curLoc = (curLat, curLong)

In [138]:
data1 = []
data2 = []
data3 = []
data4 = []

for attr in result:
    lat = float(attr['Attributes'][0]['Value']) / 1000000
    long = float(attr['Attributes'][1]['Value']) / 1000000
    dis = vincenty(curLoc, (lat, long)).miles
    data1.append(attr['Name'])
    data2.append(lat)
    data3.append(long)
    data4.append(dis)
data = DataFrame({'id': data1, 'latitude': data2, 'longitude': data3, 'distance': data4})
data.head()

,distance,id,latitude,longitude
0,38.814453,10754147,34.144442,-118.654084
1,37.152502,10759547,34.140430,-118.625364
2,22.667270,10843547,33.989359,-118.394633
3,27.093895,10859147,34.148863,-118.437206
4,25.855658,10879947,34.194168,-118.385816


In [139]:
# sort the data based on distance and pick up the closest 10 homes
data = data.sort_values(by = ['distance'], ascending=True)
data[0:10]

,distance,id,latitude,longitude
208,2.160838,13131347,34.031186,-118.003857
209,2.160838,13131547,34.031186,-118.003857
126,3.100848,13114947,34.044283,-117.990464
111,3.167951,12803347,33.980510,-118.049973
112,3.211360,12835947,34.016009,-117.947459
207,3.344965,13131147,34.047198,-118.013567
113,3.749898,12838747,34.033862,-117.948861
165,5.049879,13122747,34.015545,-118.085974
232,5.051606,13136147,34.021582,-117.915892
233,5.154032,13136347,34.024753,-117.915266
